## Chapter 10 (2)

### 2.1. Bag of words

In [1]:
sentences = [
'The stock price of google jumps on the earning data today',
'Google plunge on China Data!'
]

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
print( vectorizer.fit_transform(sentences).todense() )
print( vectorizer.vocabulary_ )

#이거 밑에 결과 해석하는거 나올수도 있을 것 같음

[[0 1 1 1 1 1 1 0 1 1 2 1]
 [1 1 0 1 0 0 1 1 0 0 0 0]]
{'the': 10, 'stock': 9, 'price': 8, 'of': 5, 'google': 3, 'jumps': 4, 'on': 6, 'earning': 2, 'data': 1, 'today': 11, 'plunge': 7, 'china': 0}


### 2.2. TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
TFIDF = vectorizer.fit_transform(sentences)

print(vectorizer.get_feature_names_out()[-10:])
print(TFIDF.shape)
print(TFIDF.toarray())

['china' 'data' 'earning' 'google' 'jumps' 'plunge' 'price' 'stock'
 'today']
(2, 9)
[[0.         0.29017021 0.4078241  0.29017021 0.4078241  0.
  0.4078241  0.4078241  0.4078241 ]
 [0.57615236 0.40993715 0.         0.40993715 0.         0.57615236
  0.         0.         0.        ]]


### 2.3. Word embedding

In [3]:
from gensim.models import Word2Vec
sentences = [
['The','stock','price', 'of', 'Google', 'increases'],
['Google','plunge',' on','China',' Data!']]

# train model
model = Word2Vec(sentences, min_count=1)

# summarize the loaded model
words = list(model.wv.index_to_key)
print(words)
print(model.wv['Google'][1:5])

['Google', ' Data!', 'China', ' on', 'plunge', 'increases', 'of', 'price', 'stock', 'The']
[ 0.00023643  0.00510335  0.00900927 -0.00930295]


In [4]:
len(model.wv['Google'])

100

### Exercise

In [6]:
import zipfile
import json

z = zipfile.ZipFile("Raw Headline Data.zip", "r")
testFile=z.namelist()[10]
fileData= z.open(testFile).read()
fileDataSample = json.loads(fileData)['content'][1:500]
fileDataSample

'li class="n-box-item date-title" data-end="1305172799" data-start="1305086400" data-txt="Tuesday, December 17, 2019">Wednesday, May 11, 2011</li><li class="n-box-item sa-box-item" data-id="76179" data-ts="1305149244"><div class="media media-overflow-fix"><div class="media-left"><a class="box-ticker" href="/symbol/CSCO" target="_blank">CSCO</a></div><div class="media-body"><h4 class="media-heading"><a href="/news/76179" sasource="on_the_move_news_fidelity" target="_blank">Cisco (NASDAQ:CSCO): Pr'

In [7]:
from lxml import etree
from io import StringIO

def jsonParser(json_data):
    xml_data = json_data['content']

    tree = etree.parse(StringIO(xml_data), parser=etree.HTMLParser())

    headlines = tree.xpath("//h4[contains(@class, 'media-heading')]/a/text()")
    assert len(headlines) == json_data['count']

    main_tickers = list(map(lambda x: x.replace('/symbol/', ''), tree.xpath("//div[contains(@class, 'media-left')]//a/@href")))
    assert len(main_tickers) == json_data['count']

    final_headlines = [''.join(f.xpath('.//text()')) for f in tree.xpath("//div[contains(@class, 'media-body')]/ul/li[1]")]
    if len(final_headlines) == 0:
        final_headlines = [''.join(f.xpath('.//text()')) for f in tree.xpath("//div[contains(@class, 'media-body')]")]
        final_headlines = [f.replace(h, '').split('\xa0')[0].strip() for f,h in zip (final_headlines, headlines)]

    return main_tickers, final_headlines

In [8]:
jsonParser(json.loads(fileData))[1][1]

'Cisco Systems (NASDAQ:CSCO) falls further into the red on FQ4 guidance of $0.37-0.39 vs. $0.42 Street consensus. Sales seen flat to +2% vs. 8% Street view. CSCO recently -2.1%.'

#### Word2Vec

In [9]:
z = zipfile.ZipFile("Raw Headline Data.zip", "r")

sentences = []
for i in range(1000):
    testFile=z.namelist()[i]
    fileData= z.open(testFile).read()
    try:
        sentences = sentences + jsonParser(json.loads(fileData))[1]
    except:
        pass

In [10]:
len(sentences)

39900

In [11]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
tokenized_sentences = [word_tokenize(i) for i in sentences]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qorud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from gensim.models import Word2Vec

# train model
model = Word2Vec(tokenized_sentences, min_count=1)

# summarize the loaded model
words = list(model.wv.index_to_key)
print(len(words))

65458


In [13]:
print(model.wv.most_similar(positive=['Apple'], topn=5))

[('Google', 0.8876566290855408), ('Amazon', 0.833220899105072), ('Microsoft', 0.8261463046073914), ('Facebook', 0.7977450489997864), ('Cisco', 0.7464077472686768)]


In [14]:
print(model.wv.most_similar(positive=['revenue'], topn=5))

[('revenues', 0.8537961840629578), ('bookings', 0.761223554611206), ('income', 0.7190226912498474), ('subscriber', 0.718623161315918), ('sales', 0.7182759046554565)]


In [15]:
print(model.wv.most_similar(positive=['firm'], topn=5))

[('company', 0.8156300783157349), ('retailer', 0.7783814668655396), ('chipmaker', 0.6567060351371765), ('agency', 0.6430360078811646), ('bank', 0.6371440887451172)]
